In [ ]:
%load_ext autoreload
%autoreload 2
import util
import scipy as sp
import numpy as np 

In [ ]:

%load_ext autoreload
%autoreload 2
import util
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np 
import engutil 
def solve_nonlinear_euler(u_signal, x0, fs, params, polys):
    """
    params = {
    'Re':  R_e,
    'Rm':  R_m,
    'Mm':  M_m,
    'R20': R_20,
    'L20': L_20

}

polys = {
    'Bl': poly_Bl,
    'K':  poly_K,
    'Le': poly_Le,
    'Li': poly_Li
}

where poly_Bl = np.poly1d(Bln)

    """
    Ts = 1.0 / fs
    num_steps = len(u_signal)
    num_states = len(x0)
    
    # Pre-allocate output
    x_history = np.zeros((num_steps, num_states))
    x_curr = x0.copy()
    
    # Extract static constants for speed
    R_e = params['Re']
    R_20 = params['R20']
    L_20 = params['L20']
    M_m = params['Mm']
    R_m = params['Rm']
    
    # Initialize Matrices (fill static parts once)
    F = np.zeros((4, 4))
    G = np.zeros(4)
    
    # Set the parts of F that NEVER change
    F[1, 0] = R_20 / L_20
    F[1, 1] = -R_20 / L_20
    F[2, 3] = 1.0
    F[3, 3] = -R_m / M_m
    
    for n in range(num_steps):
        x_history[n] = x_curr
        
        # x_curr has [i, i_L2, displacement, velocity]
        i_curr = x_curr[0]
        disp   = x_curr[2] # grab displacement value
        
        # evaluate polynomials
        val_Bl = polys['Bl'](disp)
        val_K  = polys['K'](disp)
        # displacement based inductance
        val_Le = polys['Le'](disp) 
        val_Le = max(val_Le, 1e-6) 

        # L2 is assumd to be displacement dependent like L_e, even though it is actually the current
        val_L2 = polys['Li'](disp)
        val_Le = max(val_L2, 1e-6)

        # --- 3. UPDATE MATRICES ---
        
        # Electrical Equation (Row 0)
        # di/dt = (1/Le) * (V - (Re+R20)i + R20*i2 - Bl*v)
        F[0, 0] = -(R_e + R_20) / val_Le
        F[0, 1] = R_20 / val_Le
        F[0, 3] = -val_Bl / val_Le
        
        # Mechanical Equation (Row 3)
        # dv/dt = (1/Mm) * (Bl*i - K*x - Rm*v)
        F[3, 0] = val_Bl / M_m
        F[3, 2] = -val_K / M_m
        
        # F = np.array([
        #     [-(R_e + R_20)/val_Le,  R_20/val_Le,    0.0,         -val_Bl/val_Le],
        #     [R_20/L_20,          -R_20/L_20,    0.0,          0.0 ],
        #     [0.0,                   0.0,            0.0,          1.0],
        #     [val_Bl/M_m,                0.0,           -val_K/M_m,   -R_m/M_m ]
        # ])

        # F = np.array([
        #     [-(R_e + R_20)/val_Le,  R_20/val_Le,    0.0,     -Bl/val_Le],
        #     [R_20/val_L2, -R_20/val_L2, 0.0,         0.0 ],
        #     [0.0, 0.0, 0.0, 1.0],
        #     [Bl/M_m,   0.0, -val_K/M_m,   -R_m/M_m ]
        # ])

        # Input Matrix G (Row 0 changes because of 1/Le)
        G[0] = 1.0 / val_Le
        
        # --- 4. EULER STEP ---
        # dx = F(x)*x + G(x)*u
        dx = (F @ x_curr) + (G * u_signal[n])
        
        x_next = x_curr + (dx * Ts)
        x_curr = x_next
        
    return x_history


## Params from Python driven with 3Vrms pink noise
Bl   = 6.837193705842441
R_e  = 4.822146670305587
R_m  = 2.2833222074164836
K_m  = 1107.8705431240523
L_e  = 0.000305207582223968
M_m  = 0.020004906958875274
L_20 = 0.0004325286577462285
R_20 = 2.504536634680621

Bln = [
    -2615563718.9455967,
    -3023090.98130258,
    -9835.616494456142,
    21.171849188494026, 
    Bl
]

Kn = [
    1724760519892.831,
    -5658865945.33859,
    35715360.4027017,
    -210546.86882586573,
    K_m
] 
Ln = [
    20081.188591764963,
    -319.17779393260366,
    0.8155679039802962,
    -0.01018945289267653,
    L_e
]
Li = [
    -0.0012237427320541485,
    -0.0014189369785025077,
    0.007354393550658496,
    0.005954177167801685,
    1
]
    
# p = np.poly1d([1, 2, 3])
# coeffs from Bln is Bln[0] * x**4  Bln[1] * x**3 + Bln[2] * x**2 (...) + Bl
poly_Bl = np.poly1d(Bln)
poly_K  = np.poly1d(Kn)
poly_Le = np.poly1d(Ln) 
poly_Li = np.poly1d(Li)



minimum_fs = util.calculate_min_fs(F)
print(f"Minimum fs: {minimum_fs:.2f}")
fs = int(5*minimum_fs)

# 1. create input signal
u = util.generate_pink_noise(fs*5, fs, fmin=1)

# 2. perform euler 
#x = util.solve_forward_euler(F, G, u, x0, fs)
params = {
    'Re':  R_e,
    'Rm':  R_m,
    'Mm':  M_m,
    'R20': R_20,
    'L20': L_20

}

polys = {
    'Bl': poly_Bl,
    'K':  poly_K,
    'Le': poly_Le,
    'Li': poly_Li
}


x = solve_nonlinear_euler(u, x0, fs, params, polys)
X = np.array([x[:,0], x[:,2], x[:,3]])


G_iu, G_du, G_vu, f = util.welch(u, X, fs)


G_iu_mag_db, G_iu_phase, G_iu_mag_lin  = engutil.tf_to_magphase(G_iu)
G_du_mag_db, G_du_phase, G_du_mag_lin  = engutil.tf_to_magphase(G_du)
G_vu_mag_db, G_vu_phase, G_vu_mag_lin  = engutil.tf_to_magphase(G_vu)

engutil.plot_bode(f, [(G_du_mag_db, None)], legends=["$\\hat{G}_{du}$"], title="$\\hat{G}_{du}$", ylabel_left="Magnitude $\\left| H \\right|$ / dB re 1 $\\frac{m}{V}$")
engutil.plot_bode(f, [(G_iu_mag_db, None)], legends=["$\\hat{G}_{iu}$"], title="$\\hat{G}_{iu}$", ylabel_left="Magnitude $\\left| H \\right|$ / dB re 1 S")
engutil.plot_bode(f, [(G_vu_mag_db, None)], legends=["$\\hat{G}_{vu}$"], title="$\\hat{G}_{vu}$", ylabel_left="Magnitude $\\left| H \\right|$ / dB re 1 $\\frac{m/s}{V}$")
